In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pathlib import Path

from plot import create_stacked_plot
from utils import calculate_comp_and_comm_time
from generate_comm_plot import create_combined_comm_overhead_df, get_comp_fraction_full_array
sns.set_theme()

In [3]:
logs_dir = Path("logs")
pattern = f"profile_(\w+)_(\d+)_(\d+)\.csv$"
df = create_combined_comm_overhead_df(logs_dir=logs_dir, pattern=pattern)
values = get_comp_fraction_full_array(df)
strategies = sorted(df["strategy"].unique())
gpu_numbers = sorted(df["num_gpus"].unique(), key=lambda x: int(x))

# Generating and showing the plot
fig, ax = create_stacked_plot(values, strategies, gpu_numbers)
fig.set_figwidth(8)
fig.set_figheight(6)
output_path = Path("plots/temp_plot.png")
plt.savefig(output_path)

Strategy:        ddp, Num. GPUs: 4, Comp. time: 14.85s, Comm. time: 22.34s
Strategy:        ddp, Num. GPUs: 8, Comp. time: 42.61s, Comm. time: 77.39s
Strategy:  deepspeed, Num. GPUs: 4, Comp. time: 42.20s, Comm. time:  2.18s
Strategy:  deepspeed, Num. GPUs: 8, Comp. time: 42.31s, Comm. time:  5.00s
Strategy:    horovod, Num. GPUs: 4, Comp. time: 41.96s, Comm. time: 17.66s
Strategy:    horovod, Num. GPUs: 8, Comp. time: 41.14s, Comm. time: 23.64s


In [26]:
from pprint import pprint

deepspeed_df = df[df["strategy"] == "deepspeed"]
ddp_df = df[df["strategy"] == "ddp"]
horovod_df = df[df["strategy"] == "horovod"]

deepspeed_names = set(deepspeed_df["name"].unique())
ddp_names = set(ddp_df["name"].unique())
horovod_names = set(horovod_df["name"].unique())

print(f"DeepSpeed unique entries: {len(deepspeed_names)}")
print(f"DDP unique entries: {len(ddp_names)}")
print(f"Horovod unique entries: {len(horovod_names)}")

overlapping_names = deepspeed_names.intersection(ddp_names).intersection(horovod_names)

print(f"Number of overlapping names: {len(overlapping_names)}")

pprint(horovod_names - deepspeed_names)
pprint(deepspeed_names - horovod_names)

DeepSpeed unique entries: 202
DDP unique entries: 222
Horovod unique entries: 206
Number of overlapping names: 198
{'Memcpy DtoH (Device -> Pinned)',
 'Memcpy HtoD (Pinned -> Device)',
 'aten::expand_as',
 'aten::resolve_conj',
 'aten::resolve_neg',
 'ncclKernel_AllReduce_RING_LL_Sum_float(ncclDevComm*, unsigned long, '
 'ncclWork*)',
 'ncclKernel_AllReduce_TREE_LL_Sum_float(ncclDevComm*, unsigned long, '
 'ncclWork*)',
 'void horovod::common::batched_scaled_memcpy_k<float, 8, '
 'float>(horovod::common::BatchedD2DParams, float)'}
{'c10d::broadcast_',
 'nccl:broadcast',
 'ncclKernel_Broadcast_RING_LL_Sum_int8_t(ncclDevComm*, unsigned long, '
 'ncclWork*)',
 'record_param_comms'}
